# Automatic Evaluator

In [37]:
%%capture
#########################
# Requirements and Imports
#########################
!pip install ipywidgets
!pip install widgetsnbextension

from os import path
import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

In [38]:
reproducibility_field = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)
display(reproducibility_field)

Checkbox(value=True, description='Reproducibility')

In [39]:
reproducibility = reproducibility_field.value
if reproducibility:
    seed_field = widgets.IntText(
        value=0,
        placeholder=0,
        description="Seed:",
        disabled=False
    )
    display(seed_field)

IntText(value=0, description='Seed:')

In [40]:
# Set seeds for PRGs

reproducibility = reproducibility_field.value
if reproducibility:
    seed = seed_field.value
    print(seed)
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Use deterministic Algorithms
    torch.use_deterministic_algorithms(True)
    torch.utils.deterministic.fill_uninitialized_memory = True

0


## Game Selection

In [59]:
import ocatari
all_games = ocatari.core.AVAILABLE_GAMES
print(all_games)

['Adventure', 'AirRaid', 'Alien', 'Amidar', 'Assault', 'Asterix', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'BeamRider', 'Berzerk', 'Bowling', 'Boxing', 'Breakout', 'Carnival', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'DemonAttack', 'DonkeyKong', 'DoubleDunk', 'Enduro', 'FishingDerby', 'Freeway', 'Frogger', 'Frostbite', 'Galaxian', 'Gopher', 'Hero', 'IceHockey', 'Jamesbond', 'Kangaroo', 'KeystoneKapers', 'KingKong', 'Krull', 'KungFuMaster', 'MarioBros', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Pacman', 'Phoenix', 'Pitfall', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'RoadRunner', 'Seaquest', 'Skiing', 'SpaceInvaders', 'StarGunner', 'Tennis', 'TimePilot', 'UpNDown', 'Venture', 'VideoPinball', 'YarsRevenge', 'Zaxxon']


In [60]:
hack_games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('*.py') if x.is_file() and x.name != '__init__.py']

games_option = []
for g in all_games:
    if g.lower() in hack_games:
        games_option.append(g)


game_selector = widgets.Select(
    options = games_option,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)
game_selector.layout.width = '30%'
display(game_selector)

Select(description='Game:', index=21, layout=Layout(width='30%'), options=('Amidar', 'Assault', 'Asterix', 'At…

In [80]:
selected_game = game_selector.value
selected_game

'SpaceInvaders'

# Model Selection

In [79]:
models = [m for m in Path('../models').glob('**/*.gz') if m.is_file() and selected_game.lower() in str(m).lower()]

model_selector = widgets.Select(
    options=models,
    value=None,
    description="Select Model:",
    disabled=False
)
model_selector.layout.width = '30%'
display(model_selector)


Select(description='Select Model:', layout=Layout(width='30%'), options=(WindowsPath('../models/SpaceInvaders/…

In [70]:
model_path = model_selector.value
model_path

WindowsPath('../models/SpaceInvaders/0/mdqn_modern_50M.gz')

# Add Run Arguments

In [ ]:
# Human player option only available if no model is selected
human_player = widgets.HBox([
    widgets.Text(value='--human', description='Let user play', disabled=True),
    widgets.Dropdown(options=[True, False], value=False, description='Type:', disabled=False if model_path is None else True)
])

display(human_player)

In [73]:
human_opt = '-hu' if human_player.children[1].value else ''

In [ ]:
# TODO: load from Game class
mod_commands = ['lazy_enemy', 'up_drift']
modifies = widgets.VBox([
        widgets.HBox([
            widgets.Label(value=arg),
            widgets.Dropdown(options=[True, False], value=True, description='Value:')
        ], justify_content = 'space-between') for arg in mod_commands
    ])

display(modifies)

In [76]:
mod_args = ' '.join([f'{mod_commands[i]}' if modifies.children[i].children[1].value else '' for i in range(len(mod_commands))])
mod_args

'lazy_enemy up_drift'

# Run Game

In [81]:
eval_script = Path(r"scripts/eval.py")
project_root = path.dirname(path.abspath(""))
print(f"Project Root: {project_root}")

# use posix_paths compatibile with the OS
eval_path = (Path(project_root) / eval_script).as_posix() 
model_path = Path(model_path).as_posix()

print(f"Running Game: {selected_game} with model located at: {model_path}")


Project Root: f:\Studium\TUD_WS2024\LabKI\workspace\forked\HackAtari
Running Game: SpaceInvaders with model located at: ../models/SpaceInvaders/0/mdqn_modern_50M.gz


In [82]:

command = f"python {eval_path} -g {selected_game} -a {model_path} {human_opt}"
print("command: " + command)

!{command}

command: python f:/Studium/TUD_WS2024/LabKI/workspace/forked/HackAtari/scripts/eval.py -g SpaceInvaders -a ../models/SpaceInvaders/0/mdqn_modern_50M.gz 
Loaded agent from ../models/SpaceInvaders/0/mdqn_modern_50M.gz
Episode 1: Reward = 290.0
Episode 2: Reward = 540.0
Episode 3: Reward = 365.0
Episode 4: Reward = 460.0
Episode 5: Reward = 435.0
Episode 6: Reward = 400.0
Episode 7: Reward = 205.0
Episode 8: Reward = 435.0
Episode 9: Reward = 795.0
Episode 10: Reward = 485.0

Summary:
Agent: ../models/SpaceInvaders/0/mdqn_modern_50M.gz
Total Episodes: 10
Average Reward: 441.00
Standard Deviation: 149.48
Min Reward: 205.0
Max Reward: 795.0
--------------------------------------
------------------------------------------------
Overall Average Reward: 441.00
Overall Standard Deviation: 149.48
------------------------------------------------


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]


# Evaluation